# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234065243281                   -0.50    7.0         
  2   -7.249443536050       -1.81       -1.39    1.0   8.23ms
  3   -7.250941683952       -2.82       -1.74    3.0   10.9ms
  4   -7.251162234852       -3.66       -1.97    1.0   9.00ms
  5   -7.251299512562       -3.86       -2.38    1.0   10.1ms
  6   -7.251336690203       -4.43       -2.93    2.0   11.4ms
  7   -7.251338505314       -5.74       -3.26    2.0   11.5ms
  8   -7.251338724255       -6.66       -3.74    1.0   8.69ms
  9   -7.251338789849       -7.18       -4.08    2.0   10.9ms
 10   -7.251338797459       -8.12       -4.81    1.0   9.33ms
 11   -7.251338798674       -8.92       -5.21    3.0   14.3ms
 12   -7.251338798695      -10.67       -5.56    1.0   10.3ms
 13   -7.251338798703      -11.09       -5.97    2.0   11.4ms
 14   -7.251338798704      -11.87       -6.76    1.0   9.40ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.044800525796875
[ Info: Arnoldi iteration step 2: normres = 0.3272831257206461
[ Info: Arnoldi iteration step 3: normres = 0.3462974479457861
[ Info: Arnoldi iteration step 4: normres = 0.7872913542433054
[ Info: Arnoldi iteration step 5: normres = 0.8258817411162449
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.22e-01, 1.03e-01, 5.58e-01, 2.96e-01, 2.84e-02)
[ Info: Arnoldi iteration step 6: normres = 0.24199997620542768
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.31e-01, 1.05e-01, 1.33e-01, 9.25e-02, 4.37e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09435378210200342
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.96e-03, 9.11e-03, 1.06e-02, 5.86e-02, 6.44e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13393619143177027
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.51e-04